## OR Tools Scheduling


In [1]:
from dataclasses import dataclass
from ortools.sat.python import cp_model
from typing import List
from copy import deepcopy

### Simple Test


In [2]:
@dataclass
class Topic:
    id: int
    title: str
    hours: float
    # Assigned when used in OR Tools scheduling
    assigned_day: cp_model.IntVar | int = None

@dataclass
class Plan:
    id: int
    title: str
    final_day: int
    fraction: float
    required_hours: float
    topics: List[Topic]

    def total_hours(self) -> float:
        """Sum of all the hours from each topic."""
        total = 0
        for topic in self.topics:
            total += topic.hours
        return total

In [3]:
list1 = ["a-1", "a-2"]
list2 = ["b-1", "b-2"]

zipped =list(zip(list1, list2))
print(zipped)

[('a-1', 'b-1'), ('a-2', 'b-2')]


### Functions

In [8]:
def split_topics_to_single_hours(plans: List[Plan]) -> List[Plan]:
    """
    Splits Topics that are longer than one hour to individual Topics of one hour each.
    """
    plans = deepcopy(plans)

    for plan in plans:
        split_topics = []
        for topic in plan.topics:
            while topic.hours > 0:
                split_topic: Topic = deepcopy(topic)
                split_topic.hours = 1
                split_topics.append(split_topic)
                topic.hours -= 1
        plan.topics = split_topics
    return plans

def add_assigned_day_to_topics(plans: List[Plan], model: cp_model.CpModel, n_days: int) -> None:
    """Adds the assigned_day OR Tool IntVar to each Topic in the plans."""
    id = 1 # Unique identifier for variable names
    for plan in plans:
        for topic in plan.topics:
            topic.assigned_day = model.new_int_var(0, n_days-1, str(id))
            id += 1

def add_constraints(plans: List[Plan], model: cp_model.CpModel, n_topics_per_day: List[int]) -> None:
    """
    Adds the necessary constraints to the scheduling model, ensuring that:
    1) The Topics are assigned to days in the correct order.
    2) The last Topic of each Plan is on or before the respective Plan's final day.
    3) The number of Topics assigned to a given day is equal to the required number of
       Topics for that day.
    """
    for plan in plans:
        topics = plan.topics
        # Constraining order of topics
        for i in range(len(topics)-1):
            current = topics[i]
            next = topics[i+1]
            model.add(current.assigned_day <= next.assigned_day)
        
        # Ensuring that the the last topic is on or before the final day
        last_topic = topics[-1]
        model.add(last_topic.assigned_day <= plan.final_day)

    # Adding indicators to check if a topic is scheduled on a specific day
    indicators = [
        {
            str(topic.assigned_day): model.new_bool_var(f"{topic.title}_on_day_{day}")
            for plan in plans for topic in plan.topics
        }
        for day in range(len(n_topics_per_day))
    ]

    # Setting binary indicators for use in constraining the number of topics on a given
    # day
    for day_num in range(len(indicators)):
        for id, indicator in indicators[day_num].items():
            for plan in plans:
                for topic in plan.topics:
                    if id == str(topic.assigned_day):
                        model.add(topic.assigned_day == day_num).only_enforce_if(indicator)
                        model.add(topic.assigned_day != day_num).only_enforce_if(indicator.Not())

    # Constraining such that the number of topics on each day matches the target
    for day_indicators, n_topics in zip(indicators, n_topics_per_day):
        model.add(sum(day_indicators.values()) == n_topics)

def convert_assigned_days_to_ints(plans: List[Plan], solver: cp_model.CpSolver) -> None:
    """Converts the IntVar type to integer after the model has been solved."""
    for plan in plans:
        for topic in plan.topics:
            topic.assigned_day = solver.value(topic.assigned_day)

def format_to_topic_list(plans: List[Plan], n_days: int) -> List[List[Topic]]:
    """
    Converts the list of Plans to a list of sublists where each sublist corresponds
    to a day in the study schedule and contains Topics for that day.
    """
    topics_list = [[] for _ in range(n_days)]
    for plan in plans:
        for topic in plan.topics:
            day_num = topic.assigned_day
            todays_topics = topics_list[day_num]
            is_topic_in_day = False
            for assigned_topic in todays_topics:
                if assigned_topic.id == topic.id:
                    assigned_topic.hours += 1
                    is_topic_in_day = True
                    break
            if not is_topic_in_day:
                todays_topics.append(deepcopy(topic))
    return topics_list

def create_schedule(plans: List[Plan], n_topics_per_day: List[int]) -> List[List[Topic]]:
    model = cp_model.CpModel()

    # Preparing Plan and Topic formats for compatibility in the scheduling algorithm
    # Assigning the optimizable OR Tools assigned_day field to each Topic
    topic_split_plans = split_topics_to_single_hours(plans)
    n_days = len(n_topics_per_day)
    add_assigned_day_to_topics(topic_split_plans, model, n_days)

    # Carry out the constrained optimisation
    add_constraints(topic_split_plans, model, n_topics_per_day)
    model.maximize(0) # Currently not optimising anything but this can be changed
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status != 4:
        raise Exception(
            f"The solver was not able to find a feasible solution. Error code: {status} "
        )

    # Reformatting and obtaining the schedule
    convert_assigned_days_to_ints(topic_split_plans, solver)
    schedule = format_to_topic_list(topic_split_plans, n_days)

    return schedule

    

In [17]:
# Physics
ps1 = Topic(1, "Instrumentation PS 1", 3)
topics = [ps1]
poi = Plan(3, "Maths", 2, 0.6, 6, topics)

plans = [poi]
study_hours = [
    0,
    3,
    0,
]

schedule = create_schedule(plans, study_hours)

print(schedule)

[[], [Topic(id=1, title='Instrumentation PS 1', hours=3, assigned_day=1)], []]


### Defining the Plans and Topics

In [12]:
# Physics
physics_topic_1 = Topic(1, "Physics Topic 1", 1)
physics_topics = [physics_topic_1]
physics = Plan(1, "Physics", 1, 0.1, 1, physics_topics)

# Chemistry
chemistry_topic_1 = Topic(2, "Chemistry Topic 1", 1)
chemistry_topic_2 = Topic(3, "Chemistry Topic 2", 2)
chemistry_topics = [chemistry_topic_1, chemistry_topic_2]
chemistry = Plan(2, "Chemistry", 3, 0.3, 3, chemistry_topics)


# Maths
maths_topic_1 = Topic(4, "Maths Topic 1", 1)
maths_topic_2 = Topic(5, "Maths Topic 2", 2)
maths_topic_3 = Topic(6, "Maths Topic 3", 3)
maths_topics = [maths_topic_1, maths_topic_2, maths_topic_3]
maths = Plan(3, "Maths", 3, 0.6, 6, maths_topics)

plans = [physics, chemistry, maths]

# Preparing the data structures
prepared_plans = split_topics_to_single_hours(plans)
model = cp_model.CpModel()
n_days = 3
add_assigned_day_to_topics(prepared_plans, model, n_days)

# Adding constraints
daily_hours = [5,5,0]
n_days = len(daily_hours)
indicators = add_constraints(prepared_plans, model, daily_hours)

# Solving the model
model.maximize(0) 
solver = cp_model.CpSolver()
status = solver.Solve(model)
print(status)
convert_assigned_days_to_ints(prepared_plans, solver)
list_of_topics = format_to_topic_list(prepared_plans, n_days)

for plan in prepared_plans:
    print(plan.topics)

print("")

print(list_of_topics)

4
[Topic(id=1, title='Physics Topic 1', hours=1, assigned_day=1)]
[Topic(id=2, title='Chemistry Topic 1', hours=1, assigned_day=0), Topic(id=3, title='Chemistry Topic 2', hours=1, assigned_day=0), Topic(id=3, title='Chemistry Topic 2', hours=1, assigned_day=1)]
[Topic(id=4, title='Maths Topic 1', hours=1, assigned_day=0), Topic(id=5, title='Maths Topic 2', hours=1, assigned_day=0), Topic(id=5, title='Maths Topic 2', hours=1, assigned_day=0), Topic(id=6, title='Maths Topic 3', hours=1, assigned_day=1), Topic(id=6, title='Maths Topic 3', hours=1, assigned_day=1), Topic(id=6, title='Maths Topic 3', hours=1, assigned_day=1)]

[[Topic(id=2, title='Chemistry Topic 1', hours=1, assigned_day=0), Topic(id=3, title='Chemistry Topic 2', hours=1, assigned_day=0), Topic(id=4, title='Maths Topic 1', hours=1, assigned_day=0), Topic(id=5, title='Maths Topic 2', hours=2, assigned_day=0)], [Topic(id=1, title='Physics Topic 1', hours=1, assigned_day=1), Topic(id=3, title='Chemistry Topic 2', hours=1, ass

### Week Problem

In [21]:
# Define the problem
model = cp_model.CpModel()
topics_per_day_target = [1, 1, 2, 2]
n_days = len(topics_per_day_target)
topics_to_days = {
    "Maths": model.new_int_var(0, n_days - 1, "Maths"),
    "Physics": model.new_int_var(0, n_days - 1, "Physics"),
    "Chemistry": model.new_int_var(0, n_days - 1, "Chemistry"),
    "Biology": model.new_int_var(0, n_days - 1, "Biology"),
    "English": model.new_int_var(0, n_days - 1, "English"),
    "Further Maths": model.new_int_var(0, n_days - 1, "Further Maths"),
}

# Adding indicators to check if a topic is on a specific day
indicators = [
    {
        topic_title: model.new_bool_var(f"{topic_title}_on_day_{day}")
        for topic_title in topics_to_days.keys()
    }
    for day in range(len(topics_per_day_target))
]

for day_num in range(len(indicators)):
    for topic_title, indicator in indicators[day_num].items():
        model.add(topics_to_days[topic_title] == day_num).only_enforce_if(indicator)
        model.add(topics_to_days[topic_title] != day_num).only_enforce_if(indicator.Not())

# Constraining such that the number of topics on each day matches the target
for day_indicators, n_topics in zip(indicators, topics_per_day_target):
    model.add(sum(day_indicators.values()) == n_topics)

# Define binary indicator variables
model.maximize(0)  # Dummy constraint

# Constraining the order of topics
model.add(topics_to_days["Maths"] <= topics_to_days["Physics"])
model.add(topics_to_days["Physics"] <= topics_to_days["Chemistry"])
model.add(topics_to_days["Chemistry"] <= topics_to_days["Biology"])
model.add(topics_to_days["Biology"] <= topics_to_days["English"])
model.add(topics_to_days["English"] <= topics_to_days["Further Maths"])


solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print the results
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solution found:")
    for topic, var in topics_to_days.items():
        print(f"{topic} is scheduled on day {solver.value(var)}")

    print("")
    for day_num in range(len(indicators)):
        for topic_title, indicator in indicators[day_num].items():
            print(f"{indicator}: {solver.value(indicator)}")
else:
    print("No solution found.")

Solution found:
Maths is scheduled on day 0
Physics is scheduled on day 1
Chemistry is scheduled on day 2
Biology is scheduled on day 2
English is scheduled on day 3
Further Maths is scheduled on day 3

Maths_on_day_0: 1
Physics_on_day_0: 0
Chemistry_on_day_0: 0
Biology_on_day_0: 0
English_on_day_0: 0
Further Maths_on_day_0: 0
Maths_on_day_1: 0
Physics_on_day_1: 1
Chemistry_on_day_1: 0
Biology_on_day_1: 0
English_on_day_1: 0
Further Maths_on_day_1: 0
Maths_on_day_2: 0
Physics_on_day_2: 0
Chemistry_on_day_2: 1
Biology_on_day_2: 1
English_on_day_2: 0
Further Maths_on_day_2: 0
Maths_on_day_3: 0
Physics_on_day_3: 0
Chemistry_on_day_3: 0
Biology_on_day_3: 0
English_on_day_3: 1
Further Maths_on_day_3: 1
